Asynchronous parallel processing of FVS keyfiles

In [82]:
import os
import glob
import subprocess
import ipyparallel as ipp
import shutil

Gather the list of keyfiles to run

In [104]:
run_dir = os.path.abspath('Keyfiles_to_run\\WC_rx1-3')
to_run = glob.glob(os.path.join(run_dir, '*.key'))

A function to execute FVS that will be mapped to all keyfiles.

In [86]:
def run_fvs(keyfile):
    subprocess.call(['C:\\FVSbin\\FVSwc.exe', '--keywordfile='+keyfile]) # run fvs
    # clean-up the outputs
    base_dir = os.path.split(keyfile)[0]
    base_name = os.path.split(keyfile)[-1].split('.')[0]
    # move the .out and .key file
    shutil.move(os.path.join(base_dir,base_name+'.out'), os.path.join(base_dir,'completed','outfiles'))
    shutil.move(keyfile, os.path.join(base_dir,'completed','keyfiles'))
    # delete the other files
    os.remove(os.path.join(base_dir, base_name+'.trl'))
    return keyfile

Run the following command in an ipython window to start up a cluster of workers:

`>> activate Py3.5 # or other environment name`

`>> ipcluster start -n 4 # or other number of cores`

In [91]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [5]:
# subprocess.call(['C:\\FVSbin\\FVSpn.exe', '--keywordfile='+to_run[0]])

Create a direct view of the workers and a load-balanced view for submitting jobs

In [92]:
dv = c[:]
v = c.load_balanced_view()

Import subprocess to all workers

In [93]:
with dv.sync_imports():
    import subprocess
    import shutil
    import os

importing subprocess on engine(s)
importing shutil on engine(s)
importing os on engine(s)


Execute an ayschronous batch of FVS runs for all the keyfiles

In [105]:
res = v.map_async(run_fvs, to_run)

In [98]:
res.abort()

Check on the progress

In [102]:
res.wait_interactive(10)

   2/2 tasks finished after    8 s
done


In [99]:
# returns a true/false if full set of jobs completed
res.ready()

True

In [60]:
# with database tables, load balanced map_async
print(res.wall_time)
print(res.serial_time)
print(res.serial_time/res.wall_time)

134.175687
522.9695689999998
3.8976477832381042


In [12]:
c.shutdown(hub=True)